## Risk Categorization

#### Load libaries.


In [1]:
from risk_atlas_nexus.blocks.inference import OllamaInferenceEngine
from risk_atlas_nexus.blocks.inference.params import (
    InferenceEngineCredentials,
    OllamaInferenceEngineParams,
)
from risk_atlas_nexus.library import RiskAtlasNexus

/Users/dhaval/Projects/Usage-Governance/risk-atlas-nexus/src/risk_atlas_nexus/toolkit/job_utils.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


##### Risk Atlas Nexus uses Large Language Models (LLMs) to infer risks dimensions. Therefore requires access to LLMs to inference or call the model.

**Available Inference Engines**: WML, Ollama, vLLM, RITS. Please follow the [Inference APIs](https://github.com/IBM/risk-atlas-nexus?tab=readme-ov-file#install-for-inference-apis) guide before going ahead.

_Note:_ RITS is intended solely for internal IBM use and requires TUNNELALL VPN for access.


In [2]:
inference_engine = OllamaInferenceEngine(
    model_name_or_path="granite3.2:8b",
    credentials=InferenceEngineCredentials(api_url="http://localhost:11434"),
    parameters=OllamaInferenceEngineParams(
        num_predict=1000, temperature=0, repeat_penalty=1, num_ctx=9000
    ),
)

[2025-06-11 00:35:45:843] - INFO - RiskAtlasNexus - OLLAMA inference engine will execute requests on the server at http://localhost:11434.
[2025-06-11 00:35:45:888] - INFO - RiskAtlasNexus - Created OLLAMA inference engine.


##### Create an instance of RiskAtlasNexus


In [3]:
risk_atlas_nexus = RiskAtlasNexus()

[2025-06-11 00:35:45:965] - INFO - RiskAtlasNexus - Created RiskAtlasNexus instance. Base_dir: None


#### Prepare User Intents


In [4]:
user_intent_1 = "Generate personalized, relevant responses, recommendations, and summaries of claims for customers to support agents to enhance their interactions with customers."
print("user_intent_1: ", user_intent_1)
print()
user_intent_2 = "An AI system is used by a consortium of universities and financial institutions to both assess student academic performance and determine their eligibility and risk level for student loans. The system automatically evaluates students' historical academic data, standardized test results, socio-economic background, behavioral data from educational platforms, and other digital footprints (e.g., attendance, participation, learning pace)."
print("user_intent_2: ", user_intent_2)

user_intent_1:  Generate personalized, relevant responses, recommendations, and summaries of claims for customers to support agents to enhance their interactions with customers.

user_intent_2:  An AI system is used by a consortium of universities and financial institutions to both assess student academic performance and determine their eligibility and risk level for student loans. The system automatically evaluates students' historical academic data, standardized test results, socio-economic background, behavioral data from educational platforms, and other digital footprints (e.g., attendance, participation, learning pace).


#### Invoking Risk Categorization Service on user_intent_1 and user_intent_2


In [5]:
response = risk_atlas_nexus.categorize_risk_severity(
    usecases=[user_intent_1], inference_engine=inference_engine
)

[2025-06-11 00:35:54:490] - INFO - RiskAtlasNexus - AI Domain: Customer service/support
[2025-06-11 00:36:23:738] - INFO - RiskAtlasNexus - AI User: Customer Support Agents
[2025-06-11 00:36:23:739] - INFO - RiskAtlasNexus - Purposes: To enhance customer service by providing support agents with personalized, contextually relevant information and recommendations, enabling them to address customer inquiries and claims more effectively and efficiently.
[2025-06-11 00:36:23:739] - INFO - RiskAtlasNexus - Capability: Natural Language Generation (NLG): Develop AI models to generate personalized, context-aware responses and summaries for customer interactions. 
    Sentiment Analysis: Use NLP to understand customer sentiment and tailor responses accordingly. 
    Recommendation Engine: Analyze customer data to suggest relevant products, services, or solutions based on individual preferences and needs. 
    Chatbot Integration: Integrate with customer support platforms to provide real-time, au

#### Risk Severity Information


In [6]:
response[0]

{'Description': 'The AI system intended to be used is a customer service support tool that leverages Natural Language Generation, Sentiment Analysis, a Recommendation Engine, and Chatbot Integration to provide personalized, contextually relevant information and recommendations to customer support agents, thereby enhancing their ability to address customer inquiries and claims effectively and efficiently.',
 'Classification': 'Limited Risk',
 'Relevant Text from the EU AI Act': '1. This refers to the risks associated with a need for transparency around the use of AI. The AI Act introduces specific disclosure obligations to ensure that humans are informed when necessary to preserve trust. For instance, when using AI systems such as chatbots and deepfakes, humans should be made aware that they are interacting with a machine so they can take an informed decision.',
 'Reasoning': "The AI system in question is a customer service support tool that aids customer support agents in their work. I